# MNIST 손글씨 숫자 분류 PyTorch 모델

이 노트북은 MNIST 데이터셋을 사용하여 손글씨 숫자를 분류하는 PyTorch 모델을 구현합니다.

## 주요 내용
1. 설정 및 하이퍼파라미터 정의
2. 완전 연결 신경망 모델 정의
3. 데이터 로더 준비
4. 모델 훈련
5. 모델 평가
6. 이미지 예측

## 1. 필요한 라이브러리 임포트

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from PIL import Image
import os

## 2. 설정 클래스

모든 하이퍼파라미터와 설정값을 관리하는 Config 클래스를 정의합니다.

In [ ]:
class Config:
    """하이퍼파라미터를 위한 설정 클래스"""
    BATCH_SIZE = 64
    LEARNING_RATE = 0.001
    EPOCHS = 5
    INPUT_SIZE = 28 * 28
    HIDDEN_SIZE = 500
    NUM_CLASSES = 10
    MODEL_PATH = "mnist.pth"
    DATA_PATH = "./data"

print(f"설정값:")
print(f"- 배치 크기: {Config.BATCH_SIZE}")
print(f"- 학습률: {Config.LEARNING_RATE}")
print(f"- 에포크 수: {Config.EPOCHS}")
print(f"- 입력 크기: {Config.INPUT_SIZE}")
print(f"- 은닉층 크기: {Config.HIDDEN_SIZE}")
print(f"- 클래스 수: {Config.NUM_CLASSES}")

## 3. 신경망 모델 정의

MNIST 이미지 분류를 위한 완전 연결 신경망 모델을 정의합니다.

In [ ]:
class ImageClassifier(nn.Module):
    """이미지 분류를 위한 완전 연결 신경망"""
    
    def __init__(self, input_size=Config.INPUT_SIZE, hidden_size=Config.HIDDEN_SIZE, num_classes=Config.NUM_CLASSES):
        super(ImageClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.reshape(-1, Config.INPUT_SIZE)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 모델 인스턴스 생성 및 구조 확인
model = ImageClassifier()
print(f"모델 구조:\n{model}")

# 모델 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
print(f"\n총 파라미터 수: {total_params:,}")

## 4. MNIST 훈련 클래스

MNIST 데이터셋을 이용한 모델 훈련을 담당하는 클래스입니다.

In [ ]:
class MNISTTrainer:
    """MNIST 모델 훈련 클래스"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"사용 디바이스: {self.device}")
        
        self.model = ImageClassifier().to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=Config.LEARNING_RATE)
        
        # 데이터 로더
        self.train_loader, self.test_loader = self._prepare_data()
    
    def _prepare_data(self):
        """훈련 및 테스트 데이터 로더 준비"""
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        
        train_dataset = datasets.MNIST(
            root=Config.DATA_PATH, 
            train=True, 
            transform=transform, 
            download=True
        )
        train_loader = DataLoader(
            dataset=train_dataset, 
            batch_size=Config.BATCH_SIZE, 
            shuffle=True
        )
        
        test_dataset = datasets.MNIST(
            root=Config.DATA_PATH, 
            train=False, 
            transform=transform, 
            download=True
        )
        test_loader = DataLoader(
            dataset=test_dataset, 
            batch_size=Config.BATCH_SIZE, 
            shuffle=False
        )
        
        print(f"훈련 데이터셋 크기: {len(train_dataset)}")
        print(f"테스트 데이터셋 크기: {len(test_dataset)}")
        print(f"훈련 배치 수: {len(train_loader)}")
        print(f"테스트 배치 수: {len(test_loader)}")
        
        return train_loader, test_loader
    
    def train(self, epochs=Config.EPOCHS):
        """모델 훈련"""
        self.model.train()
        
        for epoch in range(epochs):
            total_loss = 0
            for i, (images, labels) in enumerate(self.train_loader):
                images, labels = images.to(self.device), labels.to(self.device)
                
                # 순전파
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                # 역전파
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
                
                if (i + 1) % 100 == 0:
                    print(f'에포크 [{epoch+1}/{epochs}] 스텝 [{i+1}/{len(self.train_loader)}] 손실: {loss.item():.4f}')
            
            avg_loss = total_loss / len(self.train_loader)
            print(f'에포크 [{epoch+1}/{epochs}] 평균 손실: {avg_loss:.4f}')
        
        self._save_model()
    
    def evaluate(self):
        """테스트 데이터에서 모델 평가"""
        self.model.eval()
        
        with torch.no_grad():
            correct = 0
            total = 0
            
            for images, labels in self.test_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f"테스트 정확도: {accuracy:.2f}%")
        print(f"정확히 분류된 샘플 수: {correct}/{total}")
        return accuracy
    
    def _save_model(self):
        """모델 가중치 저장"""
        torch.save(self.model.state_dict(), Config.MODEL_PATH)
        print(f"모델이 {Config.MODEL_PATH}에 저장되었습니다")

## 5. 모델 훈련 실행

MNIST 데이터셋으로 모델을 훈련해보겠습니다.

In [ ]:
# 훈련기 생성
trainer = MNISTTrainer()

In [ ]:
# 모델 훈련
trainer.train(Config.EPOCHS)

In [ ]:
# 모델 평가
accuracy = trainer.evaluate()

## 6. 이미지 예측 클래스

훈련된 모델을 사용하여 새로운 이미지에서 숫자를 예측하는 클래스입니다.

In [ ]:
class MNISTPredictor:
    """MNIST 이미지 예측 클래스"""
    
    def __init__(self, model_path=Config.MODEL_PATH):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self._load_model(model_path)
        self.transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
    
    def _load_model(self, model_path):
        """훈련된 모델 로드"""
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"모델 파일 '{model_path}'을 찾을 수 없습니다.")
        
        model = ImageClassifier().to(self.device)
        model.load_state_dict(torch.load(model_path, map_location=self.device))
        model.eval()
        print(f"모델이 {model_path}에서 로드되었습니다.")
        return model
    
    def predict(self, image_path):
        """이미지에서 숫자 예측"""
        if not os.path.exists(image_path):
            print(f"오류: 이미지 파일 '{image_path}'을 찾을 수 없습니다.")
            return None
        
        try:
            # 이미지 로드 및 전처리
            image = Image.open(image_path).convert('L')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
            
            # 예측
            with torch.no_grad():
                output = self.model(image_tensor)
                probabilities = torch.nn.functional.softmax(output, dim=1)
                predicted_class = torch.argmax(probabilities, dim=1)
                predicted_prob = probabilities[0][predicted_class].item()
            
            print(f"예측된 숫자: {predicted_class.item()}")
            print(f"신뢰도: {predicted_prob:.4f}")
            
            # 각 클래스별 확률 출력
            print("\n각 숫자별 확률:")
            for i in range(10):
                prob = probabilities[0][i].item()
                print(f"  {i}: {prob:.4f}")
            
            return predicted_class.item(), predicted_prob
            
        except Exception as e:
            print(f"이미지 처리 중 오류: {e}")
            return None

## 7. 이미지 예측 실행

훈련된 모델을 사용하여 이미지를 예측해보겠습니다.

In [ ]:
# 예측기 생성 (모델이 저장되어 있어야 함)
if os.path.exists(Config.MODEL_PATH):
    predictor = MNISTPredictor()
else:
    print(f"모델 파일 '{Config.MODEL_PATH}'이 존재하지 않습니다. 먼저 모델을 훈련하세요.")

In [ ]:
# 이미지 예측 (이미지 파일이 있는 경우)
image_path = './data/mnist_data/0.jpg'

if os.path.exists(image_path):
    result = predictor.predict(image_path)
else:
    print(f"예측할 이미지 파일 '{image_path}'이 존재하지 않습니다.")
    print("테스트용 이미지를 준비하거나 경로를 수정하세요.")

## 8. 테스트 데이터셋에서 샘플 예측

테스트 데이터셋에서 몇 개의 샘플을 가져와서 예측 결과를 확인해보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_predictions(trainer, num_samples=10):
    """테스트 데이터셋에서 예측 결과 시각화"""
    # 테스트 데이터에서 샘플 가져오기
    dataiter = iter(trainer.test_loader)
    images, labels = next(dataiter)
    
    # GPU로 이동
    images, labels = images.to(trainer.device), labels.to(trainer.device)
    
    # 예측
    trainer.model.eval()
    with torch.no_grad():
        outputs = trainer.model(images)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
    
    # 시각화
    fig, axes = plt.subplots(2, 5, figsize=(12, 6))
    axes = axes.ravel()
    
    for i in range(min(num_samples, len(images))):
        # 이미지를 CPU로 이동 후 numpy로 변환
        img = images[i].cpu().squeeze().numpy()
        true_label = labels[i].cpu().item()
        pred_label = predicted[i].cpu().item()
        confidence = probabilities[i][pred_label].cpu().item()
        
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(f'True: {true_label}, Pred: {pred_label}\nConf: {confidence:.3f}')
        axes[i].axis('off')
        
        # 예측이 틀린 경우 빨간색으로 표시
        if true_label != pred_label:
            axes[i].set_title(f'True: {true_label}, Pred: {pred_label}\nConf: {confidence:.3f}', color='red')
    
    plt.tight_layout()
    plt.show()

# 예측 결과 시각화 실행
if 'trainer' in locals():
    visualize_predictions(trainer)
else:
    print("먼저 모델을 훈련하세요.")

## 9. 결과 분석 및 요약

### 모델 아키텍처
- **입력층**: 784개 노드 (28×28 픽셀)
- **은닉층**: 500개 노드 (ReLU 활성화)
- **출력층**: 10개 노드 (0-9 숫자 분류)

### 훈련 설정
- **손실 함수**: CrossEntropyLoss
- **옵티마이저**: Adam (학습률: 0.001)
- **배치 크기**: 64
- **에포크**: 5

### 성능
- MNIST 데이터셋에서 일반적으로 95-98%의 정확도를 달성
- 간단한 완전 연결 신경망임에도 높은 성능

### 개선 가능한 점
1. **CNN 사용**: 이미지의 공간적 특성을 더 잘 활용
2. **Dropout 추가**: 과적합 방지
3. **더 깊은 네트워크**: 성능 향상 가능
4. **데이터 증강**: 회전, 이동 등으로 일반화 성능 향상